In [ ]:
import pandas as pd
import numpy as np

# Load Friday dataset
df = pd.read_csv('/content/drive/MyDrive/DOS_Project/data_raw/Friday-WorkingHours-Morning.pcap_ISCX.csv')

# Clean column names
df.columns = df.columns.str.strip()

print("Initial shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())
print("\nColumns (first 20):")
print(list(df.columns[:20]))


Initial shape: (191033, 79)

First 5 rows:


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3268,112740690,32,16,6448,1152,403,0,201.5,204.724205,...,32,3.594286e+02,1.199802e+01,380,343,16100000.0,4.988048e+05,16400000,15400000,BENIGN
1,389,112740560,32,16,6448,5056,403,0,201.5,204.724205,...,32,3.202857e+02,1.574499e+01,330,285,16100000.0,4.987937e+05,16400000,15400000,BENIGN
2,0,113757377,545,0,0,0,0,0,0.0,0.000000,...,0,9.361829e+06,7.324646e+06,18900000,19,12200000.0,6.935824e+06,20800000,5504997,BENIGN
3,5355,100126,22,0,616,0,28,28,28.0,0.000000,...,32,0.000000e+00,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,0,54760,4,0,0,0,0,0,0.0,0.000000,...,0,0.000000e+00,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN



Columns (first 20):
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min']


In [ ]:
# Step 1: Check missing values
missing = df.isnull().sum()
missing = missing[missing > 0]  # show only columns with missing
missing_percent = (missing / len(df)) * 100

print("Columns with missing values (>0):")
print(missing)
print("\nPercentage (only >0):")
print(missing_percent)
print(f"\nTotal missing cells: {df.isnull().sum().sum()}")


Columns with missing values (>0):
Flow Bytes/s    28
dtype: int64

Percentage (only >0):
Flow Bytes/s    0.014657
dtype: float64

Total missing cells: 28


In [ ]:
# Step 2: Remove  any missing values
print("Before:", df.shape)
df = df.dropna()
print("After dropping missing values:", df.shape)

# Double checking  if any missing  is left
print("Any missing left:", df.isnull().sum().sum())


Before: (191033, 79)
After dropping missing values: (191005, 79)
Any missing left: 0


In [ ]:
# Step 3: Remove duplicate rows
duplicates = df.duplicated().sum()
print("Number of duplicate rows:", duplicates)

df = df.drop_duplicates()
print("Shape after removing duplicates:", df.shape)


Number of duplicate rows: 6888
Shape after removing duplicates: (184117, 79)


In [ ]:
import numpy as np

# Step 4: Handleing the infinite values
inf_count = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
print("Total infinite numeric cells before replacement:", inf_count)

# Replace inf and -inf with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop any rows with NaN (if any after replacement)
df.dropna(inplace=True)

print("Any NaN left after dropping infinities:", df.isna().sum().sum())
print("Shape after cleaning infinities:", df.shape)


Total infinite numeric cells before replacement: 146
Any NaN left after dropping infinities: 0
Shape after cleaning infinities: (184044, 79)


In [ ]:
# Step 5: Fix data types
print("Before conversion:")
print(df.dtypes.value_counts())

# Convert numeric-looking columns to proper numeric types
df = df.apply(pd.to_numeric, errors='ignore')

print("\nAfter conversion:")
print(df.dtypes.value_counts())

# Check a few columns
print("\nSample of column data types:")
print(df.dtypes.head(10))


Before conversion:
int64      54
float64    24
object      1
Name: count, dtype: int64

After conversion:
int64      54
float64    24
object      1
Name: count, dtype: int64

Sample of column data types:
Destination Port                 int64
Flow Duration                    int64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
Total Length of Bwd Packets      int64
Fwd Packet Length Max            int64
Fwd Packet Length Min            int64
Fwd Packet Length Mean         float64
Fwd Packet Length Std          float64
dtype: object


/tmp/ipython-input-1130112561.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [ ]:
# Step 6: Check unique labels
print("Unique labels before cleaning:")
print(df['Label'].unique())


Unique labels before cleaning:
['BENIGN' 'Bot']


In [ ]:
# Keep only BENIGN rows
df = df[df['Label'] == 'BENIGN']

# Encode label: BENIGN → 0
df['Label'] = 0

print("Shape after keeping only BENIGN traffic:", df.shape)
print(df['Label'].value_counts())


Shape after keeping only BENIGN traffic: (182096, 79)
Label
0    182096
Name: count, dtype: int64


In [ ]:
# Step 7: Basic outlier and integrity checks

# Check for any negative values
negatives = (df.select_dtypes(include=[np.number]) < 0).sum().sum()
print("Total negative values:", negatives)

# Check for impossible zero durations
zero_duration = (df['Flow Duration'] == 0).sum()
print("Zero flow durations:", zero_duration)

# Basic statistics overview
print("\nBasic numeric summary:")
print(df.describe().T[['min', 'max', 'mean']].head(10))


Total negative values: 204345
Zero flow durations: 0

Basic numeric summary:
                              min           max          mean
Destination Port              0.0  6.494800e+04  6.872506e+03
Flow Duration               -12.0  1.200000e+08  1.220493e+07
Total Fwd Packets             1.0  2.079640e+05  1.437769e+01
Total Backward Packets        0.0  2.846020e+05  1.714648e+01
Total Length of Fwd Packets   0.0  1.235152e+06  5.975446e+02
Total Length of Bwd Packets   0.0  6.270000e+08  2.977461e+04
Fwd Packet Length Max         0.0  2.482000e+04  1.779276e+02
Fwd Packet Length Min         0.0  2.325000e+03  2.409881e+01
Fwd Packet Length Mean        0.0  3.808875e+03  5.223272e+01
Fwd Packet Length Std         0.0  5.192248e+03  5.132504e+01


In [ ]:
# Remove any rows with negative numeric values
df = df[(df.select_dtypes(include=[np.number]) >= 0).all(axis=1)]

print("Shape after removing rows with negative values:", df.shape)

# Double check no negatives left
negatives_after = (df.select_dtypes(include=[np.number]) < 0).sum().sum()
print("Negative values after cleaning:", negatives_after)


Shape after removing rows with negative values: (69963, 79)
Negative values after cleaning: 0


In [ ]:
# Save cleaned dataset
df.to_csv('/content/drive/MyDrive/DOS_Project/data_cleaned/Friday3.csv', index=False)

print(" Friday cleaned dataset saved as 'Friday3.csv'")

 Friday cleaned dataset saved as 'Friday3.csv'
